In [1]:
!pip install -q open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 37.9 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
import torch
import torch.nn as nn
import open_clip
from PIL import Image
import os
import glob
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import zipfile

# Config khớp với Backend của bạn
MODEL_NAME = 'ViT-L-14' 
PRETRAINED = 'datacomp_xl_s13b_b90k' 
BATCH_SIZE = 512 # Tăng giảm tùy VRAM, 256 là an toàn cho P100
NUM_WORKERS = 4  # Để load ảnh nhanh hơn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Device: {device}")
print(f"   -> GPU Count: {torch.cuda.device_count()}") # Phải hiện là 2

print(f"Loading model {MODEL_NAME}...")
model, _, preprocess = open_clip.create_model_and_transforms(MODEL_NAME, pretrained=PRETRAINED)

# --- KÍCH HOẠT MULTI-GPU ---
if torch.cuda.device_count() > 1:
    print(f"⚡ Enabling DataParallel on {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)
model.eval()
print("✅ Model Loaded & Distributed!")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

🚀 Device: cuda
   -> GPU Count: 2
Loading model ViT-L-14...


open_clip_pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

⚡ Enabling DataParallel on 2 GPUs!
✅ Model Loaded & Distributed!


In [3]:
# Đường dẫn Input (Đã cập nhật theo cấu trúc thư mục trong hình)
roots = [
    # --- PART 1 ---
    '/kaggle/input/keyframe-part1/keyframes_part1_set1/keyframes', # Chứa L01, L02, L03, L04, L05
    '/kaggle/input/keyframe-part1/keyframes_part1_L06/keyframes',  # Chứa riêng L06
    
    # --- PART 2 ---
    '/kaggle/input/keyframe-part2/keyframes_part2_set1/keyframes', # Chứa L07, L08, L09, L10
    '/kaggle/input/keyframe-part2/keyframes_part2_set2/keyframes'  # Chứa L11, L12
]

all_image_paths = []

print("📂 Scanning directories...")
for root in roots:
    if not os.path.exists(root):
        print(f"⚠️ Warning: Path not found: {root}")
        continue
    
    # Tìm file jpg trong cấu trúc: Root/Lxx/Vxxx/*.jpg
    # Ví dụ: .../keyframes/L01/V001/001.jpg
    files = glob.glob(f"{root}/L*/*/*.jpg")
    print(f"   -> Found {len(files)} images in {root}")
    all_image_paths.extend(files)

print(f"🔥 TOTAL IMAGES TO ENCODE: {len(all_image_paths)}")

# Sắp xếp để xử lý tuần tự theo Batch L01 -> L12
all_image_paths.sort()

class KeyframeDataset(Dataset):
    def __init__(self, image_paths, preprocess):
        self.image_paths = image_paths
        self.preprocess = preprocess

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            image = Image.open(path).convert("RGB")
            image = self.preprocess(image)
            return image, path
        except Exception as e:
            # Trả về ảnh đen nếu lỗi file
            return torch.zeros(3, 224, 224), "ERROR"

# Lưu ý: pin_memory=True giúp đẩy dữ liệu từ CPU -> GPU nhanh hơn
dataset = KeyframeDataset(all_image_paths, preprocess)
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS, 
    pin_memory=True 
)

📂 Scanning directories...
   -> Found 165541 images in /kaggle/input/keyframe-part1/keyframes_part1_set1/keyframes
   -> Found 36892 images in /kaggle/input/keyframe-part1/keyframes_part1_L06/keyframes
   -> Found 138566 images in /kaggle/input/keyframe-part2/keyframes_part2_set1/keyframes
   -> Found 69578 images in /kaggle/input/keyframe-part2/keyframes_part2_set2/keyframes
🔥 TOTAL IMAGES TO ENCODE: 410577


In [4]:
output_dir = '/kaggle/working/embeddings'
os.makedirs(output_dir, exist_ok=True)

all_embeddings = []
all_paths = []

print(f"🚀 Starting Encoding Process with Batch Size {BATCH_SIZE}...")

# Dùng autocast để tăng tốc trên T4
with torch.no_grad(), torch.cuda.amp.autocast():
    for images, paths in tqdm(dataloader, desc="Encoding"):
        # Lọc lỗi
        valid_mask = [p != "ERROR" for p in paths]
        if not any(valid_mask): continue
        
        # Đẩy ảnh vào GPU (DataParallel sẽ tự chia việc từ đây)
        images = images[valid_mask].to(device)
        
        # Encode
        if torch.cuda.device_count() > 1:
            # Khi dùng DataParallel, gọi model.module.encode_image hoặc model(images)
            # Nhưng CLIP có hàm encode_image riêng, DataParallel bọc model lại
            # Cách gọi đúng cho DataParallel với CLIP:
            features = model.module.encode_image(images) 
        else:
            features = model.encode_image(images)
            
        features /= features.norm(dim=-1, keepdim=True)
        
        # Move to CPU
        features = features.cpu().numpy().astype(np.float16)
        
        all_embeddings.append(features)
        all_paths.extend([p for p, v in zip(paths, valid_mask) if v])

# Concatenate tất cả
final_embeddings = np.concatenate(all_embeddings, axis=0)
print(f"✅ Encoding Done. Shape: {final_embeddings.shape}")

# --- TÁCH VÀ LƯU THEO BATCH (L01, L02...) ---
print("💾 Saving to disk by Batch...")

# Tạo DataFrame tạm để group
df = pd.DataFrame({'path': all_paths})
def get_batch_id(p):
    parts = p.split('/')
    for part in parts:
        if part.startswith('L') and len(part) == 3 and part[1:].isdigit():
            return part
    return "Unknown"

df['batch'] = df['path'].apply(get_batch_id)
df['index'] = df.index # Giữ index để map với numpy array

# Group by Batch và lưu
for batch_id, group in df.groupby('batch'):
    print(f"   -> Saving Batch {batch_id} ({len(group)} items)...")
    
    # Lấy subset vectors
    indices = group['index'].values 
    batch_vectors = final_embeddings[indices]
    
    # Lấy subset paths (Cần làm sạch path để khớp với backend server)
    # Backend cần path dạng: "L01/V001/001.jpg" (Relative path)
    def clean_path(p):
        # Tìm vị trí bắt đầu của Lxx
        idx = p.find(f"/{batch_id}/")
        if idx != -1:
            return p[idx+1:] # Lấy từ L01/... trở đi
        return p
    
    cleaned_paths = group['path'].apply(clean_path).values
    
    # Lưu 2 file: .npy (vector) và .csv (metadata)
    save_base = f"{output_dir}/{batch_id}"
    np.save(f"{save_base}_vectors.npy", batch_vectors)
    
    # Lưu metadata (Video, FrameID, Path)
    meta_df = pd.DataFrame({'path': cleaned_paths})
    # Extract thêm thông tin để insert Milvus nhanh hơn
    meta_df['video'] = meta_df['path'].apply(lambda x: x.split('/')[1])
    meta_df['frame_id'] = meta_df['path'].apply(lambda x: int(x.split('/')[-1].replace('.jpg','')))
    meta_df.to_csv(f"{save_base}_meta.csv", index=False)

print("🎉 Process Completed!")

🚀 Starting Encoding Process with Batch Size 512...


/tmp/ipykernel_20/1503553522.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


Encoding:   0%|          | 0/802 [00:00<?, ?it/s]

✅ Encoding Done. Shape: (410577, 768)
💾 Saving to disk by Batch...
   -> Saving Batch L01 (32504 items)...
   -> Saving Batch L02 (34120 items)...
   -> Saving Batch L03 (31977 items)...
   -> Saving Batch L04 (33689 items)...
   -> Saving Batch L05 (33251 items)...
   -> Saving Batch L06 (36892 items)...
   -> Saving Batch L07 (32583 items)...
   -> Saving Batch L08 (35850 items)...
   -> Saving Batch L09 (34407 items)...
   -> Saving Batch L10 (35726 items)...
   -> Saving Batch L11 (33271 items)...
   -> Saving Batch L12 (36307 items)...
🎉 Process Completed!


In [5]:
%cd /kaggle/working
!zip -r embeddings.zip embeddings/

from IPython.display import FileLink
display(FileLink('embeddings.zip'))

/kaggle/working
  adding: embeddings/ (stored 0%)
  adding: embeddings/L05_meta.csv (deflated 83%)
  adding: embeddings/L04_meta.csv (deflated 83%)
  adding: embeddings/L12_meta.csv (deflated 83%)
  adding: embeddings/L02_meta.csv (deflated 83%)
  adding: embeddings/L07_meta.csv (deflated 83%)
  adding: embeddings/L10_meta.csv (deflated 83%)
  adding: embeddings/L07_vectors.npy (deflated 8%)
  adding: embeddings/L01_meta.csv (deflated 83%)
  adding: embeddings/L11_vectors.npy (deflated 8%)
  adding: embeddings/L10_vectors.npy (deflated 8%)
  adding: embeddings/L02_vectors.npy (deflated 8%)
  adding: embeddings/L09_vectors.npy (deflated 8%)
  adding: embeddings/L01_vectors.npy (deflated 8%)
  adding: embeddings/L03_meta.csv (deflated 83%)
  adding: embeddings/L06_vectors.npy (deflated 8%)
  adding: embeddings/L11_meta.csv (deflated 83%)
  adding: embeddings/L08_meta.csv (deflated 83%)
  adding: embeddings/L09_meta.csv (deflated 83%)
  adding: embeddings/L08_vectors.npy (deflated 8%)
  a

/kaggle/working/embeddings.zip